In [2]:
from system_prompt import DEFAULT_SYSTEM_PROMPT

from dotenv import load_dotenv 

from typing import Annotated, Sequence, List, Optional
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

In [3]:
load_dotenv()

True

In [4]:
config = {
    "system_prompt": DEFAULT_SYSTEM_PROMPT,
    "temperature": 0.9,
    "max_output_tokens": 1008,
    "model": "gemini-2.5-flash-lite"
}

In [5]:
class AgentState(TypedDict):
    """Represents the state of the agent's workflow."""
    # A list of messages, annotated with a reducer to append new messages
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [6]:
class InstructionStep(TypedDict):
    title: str
    details: List[str]

In [7]:
# Tools

@tool
def swap_tokens(amount: float, from_token: str, to_token: str, slippage: float = 1.0):
    """Produce a mock swap of tokens following the instructions in 'DEFAULT_SYSTEM_PROMPT'"""
    return {
        "status": "success",
        "tx_hash": "0xmockswap123",
        "details": f"Swapped {amount} {from_token} to {to_token} with {slippage}% slippage"
    }

# Mock tool to simulate sending tokens
@tool
def send_tokens(amount: float, token: str, recipient: str):
    """Produce a mock send transaction following the instructions in 'DEFAULT_SYSTEM_PROMPT '"""
    return {
        "status": "success",
        "tx_hash": "0xmocksend456",
        "details": f"Sent {amount} {token} to {recipient}"
    }

@tool
def give_instructions(
    title: str,
    summary: str,
    steps: List[InstructionStep],
    notes: Optional[List[str]] = None,
    warnings: Optional[List[str]] = None,
    checklist: Optional[List[str]] = None,
) -> str:
    """Generate step-by-step instructions in markdown format with optional notes, warnings, and checklist."""
    
    md = f"## {title}\n"
    md += f"**Short summary:** {summary}\n\n"
    md += "### Steps\n"
    for i, step in enumerate(steps, 1):
        md += f"{i}. **{step['title']}**\n"
        for d in step["details"]:
            md += f"   - {d}\n"
        md += "\n"

    if notes:
        for n in notes:
            md += f"> **Note:** {n}\n"
    if warnings:
        for w in warnings:
            md += f"> **Warning:** {w}\n"
    if checklist:
        md += "\n**Checklist**\n"
        for c in checklist:
            md += f"✅ {c}\n"

    return md

tools = [swap_tokens, send_tokens, give_instructions]


In [8]:
llm = ChatGoogleGenerativeAI(
    model=config["model"],
    temperature=config["temperature"],
    max_output_tokens=config["max_output_tokens"],
    model_kwargs={"system_instruction": config["system_prompt"]}
).bind_tools(tools)

In [9]:
MAX_CONTEXT = 16

def swap_agent(state: AgentState) -> AgentState:
    # Always include system prompt at the start
    system_prompt = SystemMessage(content=DEFAULT_SYSTEM_PROMPT)

    # Keep only the last N messages
    messages = list(state["messages"])[-MAX_CONTEXT:]

    # Call LLM with system prompt + trimmed history
    response = llm.invoke([system_prompt] + messages)

    # Just return updated messages (response may be normal text or tool call)
    return {"messages": messages + [response]}


In [10]:
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls:
        return "end"
    else: 
        return "continue"

In [11]:
# Create a new graph with the defined state
graph = StateGraph(AgentState)

# Add nodes
graph.add_node("swap_agent", swap_agent)

tool_node = ToolNode(tools=tools)
graph.add_node("tools", tool_node)

# Set entry point
graph.set_entry_point("swap_agent")

# Define edges

# The llm node can go to tool or end based on the condition
graph.add_conditional_edges(
    "swap_agent",
    should_continue,
    {
        "continue": "tools",   # if tool call, go to tools
        "end": END     # else just end (LLM reply is final)
    },
)


graph.add_edge("tools", "swap_agent")

# compile the graph
app = graph.compile()

In [12]:
def chat_with_agent(user_input: str, history=None):
    if history is None:
        history = []
    
    history.append(HumanMessage(content=user_input))
    state = {"messages": history}
    result = app.invoke(state)
    history.extend([m for m in result["messages"] if isinstance(m, AIMessage)])
    
    return history


In [14]:
history = []
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    history = chat_with_agent(user_input, history)
    print("AI:", history[-1].content)


AI: I can help you swap tokens or send tokens on the Base network. Just tell me what you need! 
AI: Got it! I can help with that.

What is the amount of USDC you'd like to send, and what's the recipient's wallet address?
AI: I've sent 5 USDC to 0xcccgtksfrj. You can view the transaction details [here](https://example.com/tx/0xmocksend456).


c:\Users\Kamiye\Desktop\swap-agent\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:2007: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


AI:  


c:\Users\Kamiye\Desktop\swap-agent\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:2007: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


AI: 


c:\Users\Kamiye\Desktop\swap-agent\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:2007: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


AI: I'm sorry, I cannot fulfill this request. The transaction could not be completed. Please try again.
